In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# $\mathbb{S}^2$ Sphere Geometry

In [1]:
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *
M.plot()

WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


2d sphere (ellipsoid parameters [ 1.  1.  1.])


In [2]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

# element, tangent vector and covector
x = np.array([0.,0.])
v = np.array([-1.,1.])
p = M.flatf(x,v)

print("x = ", x)
print("v = ", v)
print("p = ", p)

x =  [ 0.  0.]
v =  [-1.  1.]
p =  [-4.  4.]


## Riemannian Geodesics

In [ ]:
# 2nd order geodesic equation
from src.Riemannian import geodesic
geodesic.initialize(M)
xs = M.Exptf(x,v)

%matplotlib notebook
newfig()
M.plot(rotate=(30,-45))
M.plotx(xs,v,linewidth = 1.5, s=50)
plt.show()

### Geodesics from Hamiltonian equations

In [ ]:
# Hamiltonian dynamics
q = x
print(M.Hf(q,p))
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)

# geodesic
qs = M.Exp_Hamiltoniantf(q,p).T
newfig()
%matplotlib inline
M.plot(rotate=(30,-45))
M.plotx(qs,v,linewidth=1.5,s=50)
plt.show()
#plt.savefig("geodesicOnSphere.pdf")
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf(q,p) for (q,p) in zip(qs,ps)]))

# plot geodesic on R^2
%matplotlib inline
plt.figure()
M.plotR2x(qs,v)
plt.show()

## Curvature

In [3]:
from src.Riemannian import curvature
curvature.initialize(M)
# Riemannian Curvature tensor:
print("curvature = ", M.Rf(x))

curvature =  [[[[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  4.]
   [-4.  0.]]]


 [[[ 0. -4.]
   [ 4.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]]


In [ ]:
def sec_curv(x,e1,e2):
    Rm = T.tensordot(M.g(x),M.R(x), [1,0])
    sec = T.tensordot(
             T.tensordot(
                 T.tensordot(
                     T.tensordot(
                         Rm,
                         e1, [0,0]),
                     e2, [0,0]),
                 e2, [1,0]),
             e1, [0,0])
    return sec
xt = M.element()
e1 = M.vector()
e2 = M.vector()
sec_curvf = theano.function([xt,e1,e2], sec_curv(xt,e1,e2))

In [22]:
Rijk = np.tensordot(
         np.tensordot(
           np.tensordot(M.Rf(x),
                    n1, [0,0]),
                n2, [0,0]),
            n2, [1,0])
print(Rijk)
print(M.dotf(x,Rijk, n1))

[ 0.5  0. ]
1.0


In [48]:
print(M.Rf(x)[0,:,:,:])
print(M.Rf(x)[:,:,:,0])

[[[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  4.]
  [-4.  0.]]]
[[[ 0.  0.]
  [ 0. -4.]]

 [[ 0.  4.]
  [ 0.  0.]]]


In [34]:
Rm = np.tensordot(M.gf(x),M.Rf(x), [1,0])
print(Rm)
sec = np.tensordot(Rm,
                    n1, [0,0])
print(sec)

[[[[  0.   0.]
   [  0.   0.]]

  [[  0.  16.]
   [-16.   0.]]]


 [[[  0. -16.]
   [ 16.   0.]]

  [[  0.   0.]
   [  0.   0.]]]]
[[[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  8.]
  [-8.  0.]]]


In [45]:
A = np.array(np.arange(1,17)).reshape((2,2,2,2))
a = np.tensordot(A,
                   np.array([1,0]), [0,0])
print(a)
b = np.tensordot(a, np.array([1,0]), [0,0])
print(b)

[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
[[1 2]
 [3 4]]


In [28]:
Rm[0,0,:,:]

array([[ 0.,  0.],
       [ 0.,  0.]])

In [12]:
np.tensordot(M.gf(x),M.Rf(x), [1,0])

array([[[[  0.,   0.],
         [  0.,   0.]],

        [[  0.,  16.],
         [-16.,   0.]]],


       [[[  0., -16.],
         [ 16.,   0.]],

        [[  0.,   0.],
         [  0.,   0.]]]])

In [9]:
x = np.array([0.,0.])
e1 = np.array([1.,0.])
e2 = np.array([0.,1.])
be = np.vstack([e1,e2])

# Orthonormal basis under g:
n1 = M.gramSchmidt(x,be)[:,0]
n2 = M.gramSchmidt(x,be)[:,1]

In [16]:
# Sectional Curvature
print(sec_curvf(x,n1,n2))

-1.0


## Parallel Transport

In [ ]:
# Parallel transport
from src.Riemannian import parallel_transport
parallel_transport.initialize(M)

v = np.array([-1./2,-1./2])
v = v/M.normf(x,v)
t = np.linspace(0,1,n_steps.get_value())
gamma = np.vstack([t**2,-np.sin(t)]).T
dgamma = np.vstack([2*t,np.cos(t)]).T

# test parallel transport
vt = M.parallel_transportf(v,gamma,dgamma)
# plot of result
M.plot(rotate=(30,-45))
M.plotx(gamma,v=vt)
plt.show()
#plt.savefig("partrans.pdf")

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

(ts,xs) = M.Brownian_coordsf(x,dWsf(M.dim.eval()))
newfig()
M.plot()
M.plotx(xs)
plt.show()

In [ ]:
# Brownian Motion from stochastic development
from src.stochastic import Stochastic_Development
from src.stochastic import Brownian_Stochastic_Development
Brownian_Stochastic_Development.initialize(M)
# test Simulation of Brownian Motion
qsv = SD_brownian(q0)
newfig()
plotM()
plotFMx(qsv,)
plt.show()